In [ ]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd

In [ ]:
# inputs
data_path = Path("data")
stereo_seq_file = data_path / "StereoSeq" / "Mouse_brain_Adult_GEM_bin1.tsv.gz"

# outputs
ficture_path = Path("FICTURE")
transformed_data_path = ficture_path / "MouseBrain.tsv.gz"

In [ ]:
ficture_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Ficture needs sorted input and defined column names
pd.read_table(stereo_seq_file, comment="#", dtype={"geneID": "category"}).rename(
    columns={"x": "X", "y": "Y", "geneID": "gene", "MIDCounts": "Count"}
).sort_values("X").to_csv(transformed_data_path, sep="\t", index=False)

In [ ]:
conda_env = "ficture"
conda_path = "~/miniconda3/bin/activate"

conda_cmd = f"source {conda_path} {conda_env}"

In [ ]:
for n_threads, n_jobs in [(1, 1), (8, 2)]:
    outpath = ficture_path / f"minimal_nthread{n_threads}"
    ficture_cmd = (
        "ficture run_together "
        "--all "
        "--restart "
        "--dry-run "
        "--mu-scale 2 "
        "--major-axis X "
        "--n-factor 42 "
        "--train-width 15 "
        f"--in-tsv {transformed_data_path} "
        f"--out-dir {outpath} "
        f"--threads {n_threads} "
        f"--n-jobs {n_jobs} "
    )

    _ = os.popen(f"{conda_cmd} && {ficture_cmd}")

Modify the generated Makefiles by removing the non-essential parts e.g. plotting, bulk DE analysis.

In [ ]:
for n_threads, n_jobs in [(1, 1), (8, 2)]:
    outpath = ficture_path / f"minimal_nthread{n_threads}"

    ficture_cmd = f"make -f {outpath}/Makefile -j {n_jobs}"

    id_string = os.popen(
        "sbatch -J FICTURE --mem=128G -n 8 -N 1 "
        f"-o {ficture_path/'minimal_nthread'}{n_threads}.log "
        "--time=24:00:00 "
        f'--wrap="{conda_cmd} && {ficture_cmd}" '
    ).read()

    print(id_string)